<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020

<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>


# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:

- have multiple features (i.e. variables used to make the prediction),
- employ some basic feature engineering,
- work with a non-standard loss function.

Let's start with getting the data.


In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2024-10-19 16:05:37--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1 [following]
--2024-10-19 16:05:38--  https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbb2f1a9b4b800deb65e0fe0f91.dl.dropboxusercontent.com/cd/0/inline/Ccsyt7UqW18YRHuhY7DnHlCLga2nv4wNg5mzX_EUJWmBN1LbQcKhyqto5VrDHhkfNnN9hdrRgihwxrYWslFMCkmgWCt11OEEzDSPbDHbS61MMZK2J1xEk9j-gsVzt0v440g/file?dl=1# [following]
--2024-10-19 16:05:39--  https://u

In [3]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:

$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$

where $y_i$ is the ground truth, and $p_i$ is our prediction.


Let's start with implementing the loss function.


In [2]:
import numpy as np
import pandas as pd

In [118]:
# load the data from .csv files


def normalize(df: pd.DataFrame):
    return (df - df.mean()) / df.std()


def one_hot_encode(df: pd.DataFrame):
    return pd.get_dummies(df, dtype="int")


def prepare_data(df: pd.DataFrame):
    X = df.drop(columns=["cena"]).to_numpy()
    X = np.hstack(
        [X, np.ones((X.shape[0], 1))]
    )  # we're adding column of 1's for an intercept
    y = df.cena.to_numpy()

    return X, y


train_df = pd.read_csv("mieszkania.csv")
X_train, y_train = prepare_data(normalize(one_hot_encode(train_df)))

test_df = pd.read_csv("mieszkania_test.csv")
X_test, y_test = prepare_data(normalize(one_hot_encode(test_df)))

In [4]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    #################################
    # TODO: Implement this function #
    #################################

    return ((np.log(1 + ys) - np.log(1 + ps)) ** 2).mean()

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.


In [5]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

mean_price = y_train.mean()

msle(y_train, np.full(y_train.shape, mean_price))

np.float64(0.3915253538257009)

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.


In [6]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

mean_log_price = np.log(1 + y_train).mean()
optimal_const = np.exp(mean_log_price) - 1

msle(y_train, np.full(y_train.shape, optimal_const))

np.float64(0.3648896122133589)

Now, let's implement a standard linear regression model.


In [132]:
def predict(X, w):
    return X @ w


def mse_loss(y_hat, y):
    return ((y_hat - y) ** 2).mean()


def train(X, y, lr, n_epochs, predict_func=predict, loss_func=mse_loss):
    n = len(y)
    w = np.random.rand(X.shape[1])

    for _ in range(n_epochs):
        y_hat = predict_func(X, w)
        dw = 2 / n * X.T @ (y_hat - y)
        w -= lr * dw

        print(f"loss: {loss_func(y_hat, y)}")

    return w

In [134]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################
lr = 0.2
n_epochs = 100

w = train(X_train, y_train, lr, n_epochs)

y_test_predict = predict(X_test, w)

print()
msle(y_test, y_test_predict)

loss: 1.0096747736607772
loss: 0.3971510081300524
loss: 0.1851220007640192
loss: 0.10696206346829654
loss: 0.0767180751775501
loss: 0.06456974622358147
loss: 0.05952704955081982
loss: 0.057352174375376805
loss: 0.05635808821315523
loss: 0.05585837651845777
loss: 0.05556949707976662
loss: 0.05537300917643763
loss: 0.05521896217237274
loss: 0.055086068292996185
loss: 0.05496511856456366
loss: 0.05485202689380765
loss: 0.05474489605411446
loss: 0.05464277406583463
loss: 0.054545124432602704
loss: 0.05445159905122091
loss: 0.05436193994710289
loss: 0.054275936080059324
loss: 0.054193403831725454
loss: 0.05411417777246057
loss: 0.05403810596197353
loss: 0.05396504730558581
loss: 0.053894869886783256
loss: 0.05382744979784968
loss: 0.053762670250351224
loss: 0.05370042086086918
loss: 0.05364059705797942
loss: 0.053583099579745296
loss: 0.053527834042272564
loss: 0.05347471056579142
loss: 0.053423643448154756
loss: 0.053374550877847274
loss: 0.05332735468015067
loss: 0.05328198009127788
loss:

/tmp/ipykernel_12349/1277556963.py:7: RuntimeWarning: invalid value encountered in log
  return ((np.log(1 + ys) - np.log(1 + ps)) ** 2).mean()


np.float64(nan)

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?


Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>


In [41]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################


def predict_log(X, w):
    return np.log(1 + np.maximum(predict(X, w), 0))


lr = 10000
n_epochs = 50

w = train(X_train, np.log(1 + y_train), lr, n_epochs, predict_log, msle)

y_test_predict = predict(X_test, w)

print()
msle(y_test, y_test_predict)

loss: 5.370415607087725
loss: 0.0032922742012804597
loss: 0.0025291894161231197
loss: 0.002063336397738741
loss: 0.0017534847533946891
loss: 0.001532919289518711
loss: 0.001366021381717702
loss: 0.0012328401776769532
loss: 0.0011220423646839631
loss: 0.0010271405722058293
loss: 0.0009443408651192816
loss: 0.0008713144559043411
loss: 0.0008065182694681497
loss: 0.0007488338151805821
loss: 0.0006973822147068065
loss: 0.0006514319168347552
loss: 0.0006103529070979834
loss: 0.0005735932376995219
loss: 0.0005406658564137322
loss: 0.0005111400463708956
loss: 0.0004846349266875116
loss: 0.00046081394425054296
loss: 0.0004393799472828574
loss: 0.0004200707041299054
loss: 0.0004026548297483451
loss: 0.0003869281089860131
loss: 0.0003727102057715122
loss: 0.0003598417402259265
loss: 0.0003481817088526288
loss: 0.00033760521852583576
loss: 0.00032800150310125913
loss: 0.0003192721915630042
loss: 0.00031132979804908786
loss: 0.0003040964063116126
loss: 0.0002975025237590977
loss: 0.000291486082921

np.float64(0.08029247505795575)

Without any feature engineering our model approximates the price as a linear combination of original features:

$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$

Let's now introduce some interactions between the variables. For instance, let's consider a following formula:

$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$

Here, we model the price with far greater granularity, and we may expect to see more acurate results.


Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.


Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>


Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>


In [122]:
###############################################
# TODO: Implement the feature engineering part #
###############################################


def add_new_features(df: pd.DataFrame):
    new_df = df.copy()

    new_df["dzielnica_grodziskowo_m2"] = new_df.m2 * new_df.dzielnica_grodziskowo
    new_df["dzielnica_mokotowo_m2"] = new_df.m2 * new_df.dzielnica_mokotowo
    new_df["dzielnica_ochotowo_m2"] = new_df.m2 * new_df.dzielnica_ochotowo
    new_df["dzielnica_wolowo_m2"] = new_df.m2 * new_df.dzielnica_wolowo

    return new_df


X_train_v2, y_train_v2 = prepare_data(
    add_new_features(normalize(one_hot_encode(train_df)))
)
X_test_v2, y_test_v2 = prepare_data(
    add_new_features(normalize(one_hot_encode(test_df)))
)

In [139]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

lr = 0.01
n_epochs = 100000

w = train(X_train_v2, y_train_v2, lr, n_epochs)

y_test_predict = predict(X_test_v2, w)

print()
msle(y_test_v2, y_test_predict)

loss: 2.1237980827016556
loss: 2.0257210678549797
loss: 1.9328214148062337
loss: 1.8448001216893277
loss: 1.761377306399711
loss: 1.6822908270313968
loss: 1.6072950152421897
loss: 1.5361595122342235
loss: 1.468668198056289
loss: 1.4046182058479562
loss: 1.3438190134643542
loss: 1.2860916056549114
loss: 1.2312677006282144
loss: 1.1791890354266101
loss: 1.129706705065259
loss: 1.082680550867532
loss: 1.0379785938575763
loss: 0.9954765094566406
loss: 0.9550571400768866
loss: 0.9166100425189657
loss: 0.8800310673612057
loss: 0.845221967782077
loss: 0.8120900354865348
loss: 0.7805477616134882
loss: 0.7505125206882515
loss: 0.7219062758525265
loss: 0.6946553037569921
loss: 0.6686899376396602
loss: 0.6439443272382056
loss: 0.6203562142978276
loss: 0.597866722539027
loss: 0.5764201610430318
loss: 0.555963840097418
loss: 0.5364478986215933
loss: 0.5178251423620038
loss: 0.5000508921108482
loss: 0.4830828412603705
loss: 0.46688092205797466
loss: 0.4514071799759648
loss: 0.43662565565410444
loss:

/tmp/ipykernel_12349/1277556963.py:7: RuntimeWarning: invalid value encountered in log
  return ((np.log(1 + ys) - np.log(1 + ps)) ** 2).mean()


np.float64(nan)

In [140]:
for y_hat, y in zip(y_test_predict, y_test_v2):
    print(f"Real price: {y}, predicted price: {y_hat}")

Real price: -0.8068922750582344, predicted price: -0.4954250587368499
Real price: -0.8976708006415216, predicted price: -0.8591994585482192
Real price: -0.8609721458723396, predicted price: -0.9732461333276801
Real price: -0.009616411863014934, predicted price: -0.10216164138440037
Real price: 1.4777161223502073, predicted price: 1.6490238751645763
Real price: 0.4798999571643246, predicted price: 0.6693402259316946
Real price: -1.5733746181326362, predicted price: -1.5259988277302081
Real price: 1.153564194893577, predicted price: 1.083355851629054
Real price: -0.06867464881882768, predicted price: -0.2186761782434043
Real price: -0.10108053942706328, predicted price: -0.11183393307598605
Real price: 1.0509340571805503, predicted price: 0.9167797702187523
Real price: 0.49969628370429076, predicted price: 0.4857419891177071
Real price: -0.4363605794357454, predicted price: -0.19724660092221052
Real price: -0.02999842853729963, predicted price: -0.05431160237956961
Real price: 0.38975190